# Theory behind this transformation
### See pp.186-187 of D'Alessandro

We are interested in 2-qubit unitaries. Suppose that we ignore the global phase. Then w.l.o.g. our target unitary is a special unitary in $\mathsf{SU}(4)$. The corresponding Lie algebra is $\mathsf{su}(4)$.

We have a constand drift Hailtonian $H_D = \sigma_{z}\otimes\sigma_{z}$, whereas we can arbitrarily control the local Hamiltonians $H_C = \sum_{i=x,y,z}(a_{i}(t)\sigma_{i}\otimes I + b_i(t)I\otimes\sigma_{i})$ and coefficients $a_{i}(t),b_{i}(t)$ can be arbitrarily strong.

Then we naturally obtain the Cartan decomposition 
\begin{equation}
    \mathsf{su}(4) = \mathfrak{k} \oplus \mathfrak{p},
\end{equation}
where the subalgebra 
\begin{equation}
    \mathfrak{k} = \mathrm{span}\{i\sigma_{j}\otimes I, iI\otimes\sigma_{j} \vert j = x,y,z\}, 
\end{equation}
and
\begin{equation}
    \mathfrak{p} = \mathrm{span}\{i\sigma_{j}\otimes \sigma{k} \vert j,k = x,y,z\}.
\end{equation}
Note that 
\begin{equation}
    [ \mathfrak{k},\mathfrak{k} ] \subseteq \mathfrak{k},\quad [ \mathfrak{k},\mathfrak{p} ] \subseteq \mathfrak{p},\quad [ \mathfrak{p},\mathfrak{p} ] \subseteq \mathfrak{k}.
\end{equation}
The Lie group corresponding to the Lie algebra $\mathfrak{k}$ is $\mathcal{K} = \exp(\mathfrak{k})$.

Now we can define a Cartan subalgebra (included in $\mathfrak{p}$) w.r.t. the decomposition
\begin{equation}
    \mathfrak{a} = \mathrm{span}\{i\sigma_j\otimes\sigma_j \vert j = x,y,z\}.
\end{equation}
Then a decomposition 
\begin{equation}
    U = K_{1}AK_{2},
\end{equation}
where $K_1,K_2\in\mathcal{K}$ and $A \in \exp(\mathfrak{a})$ always exists. 
By finding the exponent of $A$, one can find the minimal time it takes to generate the target unitary $U$. 

It is known that $\mathfrak{k} = T \mathsf{so}(4) T^{-1}$, where $T$ is defined in the code below.
Hence, we first transform our unitary to be 
\begin{equation}
    \tilde{U} = T^{-1}UT = (T^{-1}K_1 T) (T^{-1} A T) (T^{-1} K_2 T) = O_1 D O_2,
\end{equation}
where $O_1, O_2$ are real orthogonal matrices with determinant 1 and $D$ is a diagonal matrix. 

Hence, if we find $D$, we can apply the inverse tranformation to get $A = TDT^{-1}$.

In [1]:
import numpy as np
from scipy.linalg import eig, det, sqrtm, inv, logm

# Groundworks

## Define $T$, $T^{-1}$, and a function to output the adjoint $T^{-1}UT$ (forward) or $TUT^{-1}$ (backward)

In [2]:
T = np.sqrt(1/2)*np.array([[0,1j,1,0],[1j,0,0,-1],[1j,0,0,1],[0,-1j,1,0]])
Tinv = np.transpose(np.conjugate(T))


def Tconj(mat,d = 'f'):
    if d=='f':
        return Tinv@mat@T
    else:
        return T@mat@Tinv

## Convert a general (4-dimensional) unitary into a special unitary 
### It only changes the global phase of a state which we do not care. If we need to care, use factor output

In [3]:
def UtoSU(mat):
    factor = np.log(det(mat),dtype = complex)/4
    newmat = mat/np.power(det(mat),1/4,dtype = complex)
    return newmat, factor

## Finding $O_1$ and $D$ 

We have $\tilde{U} = O_1DO_2$. Then $\tilde{U}\tilde{U}^{\top} = O_1D^2O_1^{\top}$.
Equivalently,
\begin{equation}
    \tilde{U}\tilde{U}^{\top} O_1 = O_1 D^2.
\end{equation}

Then $O_1$ is a matrix that has real orthonormal eigenvectors of $\tilde{U}\tilde{U}^{\top}$ as its columns and $D^2$ is a diagonal matrix whose diagonal entries are eigenvalues of $\tilde{U}\tilde{U}^{\top}$.

## This part is problematic!!! We need a better algorithm for $U = O_1DO_2$.

We cannot really recover $D$ that makes $O_2$ real and orthogonal.
See
\begin{equation}
D = \begin{pmatrix}
    1 & 0 & 0 & 0\\
    0 & 1 & 0 & 0\\
    0 & 0 & 1 & 0\\
    0 & 0 & 0 & -1
\end{pmatrix}
\end{equation}
which gives $D^2 = I$. Then the sqrtm function will give $D = I$ wrongly.

This wrong evaluation makes $\mathrm{det}(O_2) = -1$, and we don't want that. In such cases, we simply multiply -1 to the first diagonal element of $D$ and the first row of $O_2$. This makes $\mathrm{det}(O_2) = 1$. 

In [4]:
# from U, compute UU^{T}
def UUT(mat):
    return mat@np.transpose(mat)

In [5]:
# finding O_1 and D^2
def O1D2(UUTmat):
    w, v = eig(UUTmat)
    Diagmat = np.diag(w)
    return v, Diagmat

In [6]:
# output a candidate for O_1, D, O_2
def ODO_decomp(U_target):
    Utilde, factor = UtoSU(Tconj(U_target))
    O1, D2 = O1D2(UUT(Utilde))
    D1 = sqrtm(D2)
    O2 = inv(O1@D1)@Utilde
    if det(O2) != 1:
        D1 = np.diag([-1,1,1,1])@D1
        O2 = np.diag([-1,1,1,1])@O2
    return O1, D1, O2, factor

def KAK_decomp(U_target):
    O1, D, O2 , factor = ODO_decomp(U_target)
    return Tconj(O1,d = 'r'), Tconj(D,d = 'r'), Tconj(O2,d = 'r'), factor

From $A\in\exp(\mathfrak{a})$, 
\begin{equation}
    A = e^{ia} = e^{i(c_x \sigma_x\otimes\sigma_x +c_y \sigma_y\otimes\sigma_y + c_z \sigma_z\otimes\sigma_z)}.
\end{equation}
The minimal time is $c_x+c_y+c_z$.

To find $c_i$, use the inner product:
\begin{equation}
    c_i = \mathrm{Tr}[a(\sigma_i\otimes\sigma_i)]/4.
\end{equation}

In [7]:
sigx2 = np.kron(np.array([[0,1],[1,0]]),np.array([[0,1],[1,0]]))
sigy2 = np.kron(np.array([[0,-1j],[1j,0]]),np.array([[0,1],[1,0]]))
sigz2 = np.kron(np.array([[1,0],[0,-1]]),np.array([[1,0],[0,-1]]))
def optimal_drift(A):
    a = logm(A)/1j
    c_x = np.trace(a@sigx2)/4
    c_y = np.trace(a@sigy2)/4
    c_z = np.trace(a@sigz2)/4
    return c_x,c_y,c_z,np.abs(c_x)+np.abs(c_y)+np.abs(c_z)

## Example 

In [8]:
CZ = np.diag([1,1,1,-1])
K1, A, K2, factor = KAK_decomp(CZ)
cx, cy, cz, tmin = optimal_drift(A)
print('mintime: ',tmin)
print('drift term:', cx, ' for XX', cy, ' for YY', cz,' for ZZ')
print('control unitary before drift: \n', K1)
print('control unitary after drift: \n', K2)

mintime:  1.5707963267948961
drift term: (-0.7853981633974481-2.1510571102112408e-16j)  for XX (-5.704771009112362e-17-5.551115123125783e-17j)  for YY (0.7853981633974481-6.938893903907228e-18j)  for ZZ
control unitary before drift: 
 [[ 0.00000000e+00+5.00000000e-01j -2.39066301e-01-9.80444928e-18j
   2.39066301e-01+9.80444928e-18j  0.00000000e+00+5.00000000e-01j]
 [ 2.56291923e-17+5.00000000e-01j  5.00000000e-01-1.18350172e-17j
   5.00000000e-01+1.18350172e-17j  2.98819589e-17-5.00000000e-01j]
 [-2.98819589e-17-5.00000000e-01j  5.00000000e-01+1.18350172e-17j
   5.00000000e-01-1.18350172e-17j -2.56291923e-17+5.00000000e-01j]
 [ 0.00000000e+00-5.00000000e-01j -6.65467733e-01+9.80444928e-18j
   6.65467733e-01-9.80444928e-18j  0.00000000e+00-5.00000000e-01j]]
control unitary after drift: 
 [[ 7.35702260e-01-2.77555756e-17j  5.55111512e-17-5.00000000e-01j
  -5.55111512e-17+5.00000000e-01j  2.64297740e-01-2.77555756e-17j]
 [-5.42591491e-50-5.52770798e-01j -5.00000000e-01+2.12638329e-18j
  

In [9]:
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
K1, A, K2, factor = KAK_decomp(CNOT)
cx, cy, cz, tmin = optimal_drift(A)
print('mintime: ',tmin)
print('drift term:', cx, ' for XX', cy, ' for YY', cz,' for ZZ')
print('control unitary before drift: \n', K1)
print('control unitary after drift: \n', K2)

mintime:  0.7853981633974483
drift term: (-0.7853981633974482+4.163336342344337e-17j)  for XX (6.938893903907228e-17-1.1102230246251565e-16j)  for YY -2.7755575615628907e-17j  for ZZ
control unitary before drift: 
 [[ 1.66533454e-16+1.12589471e-16j  1.11022302e-16+1.83284050e-17j
   7.07106781e-01-2.41867861e-17j -7.07106781e-01-4.80856021e-17j]
 [ 5.55111512e-17-2.38752869e-18j  0.00000000e+00+5.47247384e-18j
  -7.07106781e-01-9.42717060e-18j -7.07106781e-01+1.56716807e-18j]
 [ 7.07106781e-01+1.56716807e-18j -7.07106781e-01+9.42717060e-18j
   0.00000000e+00-5.47247384e-18j -5.55111512e-17-2.38752869e-18j]
 [-7.07106781e-01-6.29367004e-17j -7.07106781e-01+3.13243651e-17j
  -1.11022302e-16-3.71827462e-17j  1.66533454e-16+1.09455134e-16j]]
control unitary after drift: 
 [[ 5.34680621e-17-5.55111512e-17j  6.73458499e-17+0.00000000e+00j
   2.80935384e-17-7.07106781e-01j -9.45364669e-18+7.07106781e-01j]
 [-7.07106781e-01-1.90980165e-17j  7.07106781e-01+2.24653325e-17j
  -1.11022302e-16-1.41

In [10]:
SWAP = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
K1, A, K2, factor = KAK_decomp(SWAP)
cx, cy, cz, tmin = optimal_drift(A)
print('mintime: ',tmin)
print('drift term:', cx, ' for XX', cy, ' for YY', cz,' for ZZ')
print('control unitary before drift: \n', K1)
print('control unitary after drift: \n', K2)

mintime:  1.570796326794897
drift term: (0.7853981633974485-1.3877787807814457e-17j)  for XX (-9.71445146547012e-17+5.551115123125783e-17j)  for YY (-0.7853981633974484-3.122502256758253e-17j)  for ZZ
control unitary before drift: 
 [[-0.20412415+0.59229013j -0.63784254+0.01442622j  0.63784254-0.01442622j
   0.20412415+0.18404184j]
 [-0.09158   +0.10668525j  0.55901586-0.03016342j  0.44098414+0.03016342j
  -0.09158   -0.70981133j]
 [ 0.09158   -0.10668525j  0.44098414+0.03016342j  0.55901586-0.03016342j
   0.09158   +0.70981133j]
 [ 0.20412415-0.59229013j -0.28975034-0.01442622j  0.28975034+0.01442622j
  -0.20412415-0.18404184j]]
control unitary after drift: 
 [[-0.15728869-4.49765972e-01j  0.24062072+1.49535502e-01j
  -0.24062072-1.49535502e-01j  0.28714375+9.57149330e-01j]
 [-0.53902958+6.63513899e-19j -0.5       +9.57738001e-20j
  -0.5       -9.57738001e-20j -0.53902958-1.28101650e-18j]
 [ 0.53902958+6.63513899e-19j -0.5       +9.57738001e-20j
  -0.5       -9.57738001e-20j  0.539029

In [11]:
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import *
H = SymbolicHamiltonian( X(1)*X(2) + Y(1)*Y(2)+Z(1)*Z(2)+Z(1))
K1, A, K2, factor = KAK_decomp(CNOT)
cx, cy, cz, tmin = optimal_drift(A)
print('mintime: ',tmin)
print('drift term:', cx, ' for XX', cy, ' for YY', cz,' for ZZ')
print('control unitary before drift: \n', K1)
print('control unitary after drift: \n', K2)

[Qibo 0.2.9|INFO|2024-07-11 14:05:15]: Using qibojit (numba) backend on /CPU:0


mintime:  0.7853981633974483
drift term: (-0.7853981633974482+4.163336342344337e-17j)  for XX (6.938893903907228e-17-1.1102230246251565e-16j)  for YY -2.7755575615628907e-17j  for ZZ
control unitary before drift: 
 [[ 1.66533454e-16+1.12589471e-16j  1.11022302e-16+1.83284050e-17j
   7.07106781e-01-2.41867861e-17j -7.07106781e-01-4.80856021e-17j]
 [ 5.55111512e-17-2.38752869e-18j  0.00000000e+00+5.47247384e-18j
  -7.07106781e-01-9.42717060e-18j -7.07106781e-01+1.56716807e-18j]
 [ 7.07106781e-01+1.56716807e-18j -7.07106781e-01+9.42717060e-18j
   0.00000000e+00-5.47247384e-18j -5.55111512e-17-2.38752869e-18j]
 [-7.07106781e-01-6.29367004e-17j -7.07106781e-01+3.13243651e-17j
  -1.11022302e-16-3.71827462e-17j  1.66533454e-16+1.09455134e-16j]]
control unitary after drift: 
 [[ 5.34680621e-17-5.55111512e-17j  6.73458499e-17+0.00000000e+00j
   2.80935384e-17-7.07106781e-01j -9.45364669e-18+7.07106781e-01j]
 [-7.07106781e-01-1.90980165e-17j  7.07106781e-01+2.24653325e-17j
  -1.11022302e-16-1.41

In [59]:
from qibo.transpiler.unitary_decompositions import two_qubit_decomposition,cnot_decomposition

Z = np.array([[1,0],[0,-1]])
X = np.array([[0,1],[1,0]])
id = np.array([[1,0],[0,1]])

operator = 1/np.sqrt(2) * (np.kron(Z,Z)+np.kron(X,id))
print(operator)
two_qubit_decomposition(0,1,operator)
import scipy
u = 1j* scipy.linalg.expm( -1j * 0.1 * operator)
print(u)

two_qubit_decomposition(0,1,u)

[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.         -0.70710678  0.          0.70710678]
 [ 0.70710678  0.         -0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]]
[[ 0.07059289+0.99500417j  0.        +0.j          0.07059289+0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.07059289+0.99500417j  0.        +0.j
   0.07059289+0.j        ]
 [ 0.07059289+0.j          0.        +0.j         -0.07059289+0.99500417j
   0.        +0.j        ]
 [ 0.        +0.j          0.07059289+0.j          0.        +0.j
   0.07059289+0.99500417j]]


In [72]:
#D = np.outer(rho_A,rho_A)
state_01 = np.array([0,1,0,0])
state_10 = np.array([0,0,1,0])
operator_0 = np.outer(state_10,state_01)
operator_1 = np.outer(state_01,state_10)
N = 2*operator_0 + 2*operator_1
print(N)
t = 0.1
N_unitary = scipy.linalg.expm(
    -1j * N * t
)
two_qubit_decomposition(0,1,unitary=N_unitary)

[[0 0 0 0]
 [0 0 2 0]
 [0 2 0 0]
 [0 0 0 0]]
